In [7]:
import torch
from torch import nn
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, Resize, ToTensor
from torchvision.datasets import ImageFolder
import torch.nn.functional as F


In [3]:
# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 53 * 53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 53 * 53)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate the model
model = SimpleCNN()

# Choose the hyperparameters for training: 
num_epochs = 10
batch_size = 64

# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Optimizer and loss function
optimizer = Adam(model.parameters())
criterion = CrossEntropyLoss()

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print("Device: ", device)

model = model.to(device)

transform = Compose([Resize((224,224)), ToTensor()])

# Prepare the datasets and loaders
train_dataset = ImageFolder(root='waste/train', transform=transform)
test_dataset = ImageFolder(root='waste/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


Device:  cpu


In [8]:
# Training Loop
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    # print statistics
    print(f"Epoch {epoch + 1} loss: {running_loss / len(train_loader)}")

print('Finished Training')

Epoch 1 loss: 0.4673615046921938
Epoch 2 loss: 0.40646668977338934
Epoch 3 loss: 0.3611138430094246
Epoch 4 loss: 0.31061740835901697
Epoch 5 loss: 0.24676357447130146
Epoch 6 loss: 0.1617218485423931
Epoch 7 loss: 0.0919192249968479
Epoch 8 loss: 0.0575358638897401
Epoch 9 loss: 0.05424362372227076
Epoch 10 loss: 0.029829617676116717
Finished Training


In [9]:
# Set the model to evaluation mode
model.eval()

correct = 0
total = 0

# Since we're testing the model we don't need to perform backprop
with torch.no_grad():
    for images, labels in test_loader:
        # Move the images and labels to the GPU if one is available
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct / total}%')

Accuracy of the network on the test images: 87.30600875447672%


In [10]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import numpy as np

# initialize lists to hold model outputs and targets
all_preds = []
all_labels = []

# again, since we're testing the model we don't need to perform backprop
with torch.no_grad():
    for images, labels in test_loader:
        # move the images and labels to the GPU if one is available
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        # collect all model outputs and targets
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# compute the confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print('Confusion Matrix:')
print(cm)

# compute precision, recall and f1 score
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


Confusion Matrix:
[[1336   65]
 [ 254  858]]
Precision: 0.8797781908775405
Recall: 0.8730600875447672
F1 Score: 0.8711759740131764


In [11]:
torch.save(model.state_dict(), 'waste_classifier.pth')

In [14]:
from ipywidgets import FileUpload

upload = FileUpload()
upload


FileUpload(value={}, description='Upload')

In [21]:
from PIL import Image
import io
import torchvision.transforms as transforms

# Take the first uploaded file
uploaded_filename = next(iter(upload.value))

# Get the file content
content = upload.value[uploaded_filename]['content']

# Convert the bytes to a PIL Image object
image = Image.open(io.BytesIO(content))

# Transform the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

image_tensor = transform(image)

# Add an extra batch dimension
image_tensor = image_tensor.unsqueeze(0)


In [22]:
model.eval()

with torch.no_grad():
    # Move the image tensor to the GPU if one is available
    image_tensor = image_tensor.to(device)
    output = model(image_tensor)
    _, prediction = torch.max(output.data, 1)

if prediction.item()==1:
    print('Predicted as Recycle Waste')
else:
    print('Predicted as Organic Waste')


Predicted as Organic Waste
